<a href="https://colab.research.google.com/github/mdkamrulhasan/data_mining_kdd/blob/main/notebooks/Association_rules_mining_market_basket_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Summary:
* [Problem definition](#problem_definition)
* [Data Analysis](#data_analysis)
* [Market Basket Analysis](#market_basket_analysis)
    * [Item Recommendation](#item_recommendation)
* [Conclusions](#conclusions)
* [References](#references)

[ref](https://www.kaggle.com/code/ceruttivini/association-rules-mining-market-basket-analysis/notebook)

In [ ]:
!pip install unidecode

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.7 MB/s eta 0:00:00


[MLxtend](https://rasbt.github.io/mlxtend/)

Mlxtend (machine learning extensions) is a Python library of useful tools for the day-to-day data science tasks.

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import unidecode
import plotly.express as px

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<a id='problem_definition'></a>
### Problem Defination:

This dataset consisted of 33k orders from two Indian takeaway restaurants in London, UK. The purpose of this notebook is to increase cross-selling when the customer performs by applying makert basket analysis (association rules) where the food could be or recommend when the takeaway is performed:

* Both X and Y could be placed on the same shelf, so that buyers of one item would be prompted to buy the other;
* Promotional discounts could be applied to only one of the two items;
* Advertisements on X could be targeted to shoppers buying Y;
* X and Y could be combined into a new product.

<a id='data_analysis'></a>
### Data Analysis:

Since we have two data sets, we need to merge the information into one for easier analysis. To differentiate it between restaurant orders, first and second, a specific column was created for this purpose and remove the total_products column because it is a sum of the quantity of items in each order.

In [ ]:
data_url = 'https://raw.githubusercontent.com/mdkamrulhasan/data_mining_kdd/main/data/association/restaurant-1-orders.csv'
df = pd.read_csv(data_url)

In [ ]:
orders_first_restaurant = pd.read_csv('https://raw.githubusercontent.com/mdkamrulhasan/data_mining_kdd/main/data/association/restaurant-1-orders.csv')
orders_first_restaurant['restaurant'] = '1 - restaurant'

orders_second_restaurant = pd.read_csv('https://raw.githubusercontent.com/mdkamrulhasan/data_mining_kdd/main/data/association/restaurant-2-orders.csv')
orders_second_restaurant['restaurant'] = '2 - restaurant'
orders_second_restaurant.rename(columns={'Order ID':'Order Number'},inplace=True)

orders = pd.concat([orders_first_restaurant,orders_second_restaurant])
orders.drop('Total products',axis=1,inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


To facilitate the analysis, we formatted the columns in lowercase and without spaces.

In [ ]:
orders.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 194001 entries, 0 to 119182
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Order Number   194001 non-null  int64  
 1   Order Date     194001 non-null  object 
 2   Item Name      194001 non-null  object 
 3   Quantity       194001 non-null  int64  
 4   Product Price  194001 non-null  float64
 5   restaurant     194001 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 10.4+ MB


In [ ]:
orders.sample(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Order Number,Order Date,Item Name,Quantity,Product Price,restaurant
74527,3880,02/12/2016 20:24,King Prawn Shaslick,1,12.95,1 - restaurant
74685,2095,18/03/2016 17:51,King Prawn Shaslick,1,12.95,1 - restaurant
22294,9346,24/04/2018 17:59,Pilau Rice,2,2.95,1 - restaurant
6517,12557,28/10/2017 19:17,Onion Chutney,1,0.50,2 - restaurant
83200,25356,24/07/2019 19:52,Vindaloo - Chicken,1,7.95,2 - restaurant


In [ ]:
def format_columns(column):
    new_column = ' '.join(column.split())
    new_column = new_column.replace(' ','_')
    return new_column.lower()

orders.columns = [format_columns(c) for c in orders.columns]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


How we can see, there is no null values in orders

In [ ]:
orders.isna().mean()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


order_number     0.0
order_date       0.0
item_name        0.0
quantity         0.0
product_price    0.0
restaurant       0.0
dtype: float64

In [ ]:
def stats_summary(data):

    sales = data['product_price'].sum()
    quantity = data['quantity'].sum()
    orders_count = len(data['order_number'].unique())

    avg_price_order = sales / orders_count
    avg_price_food = sales / quantity
    count_food_orders = quantity / orders_count

    return {'avg_price_order': avg_price_order,
            'avg_price_food': avg_price_food,
            'count_food_orders': count_food_orders
           }

def print_summary(data):

    stats = stats_summary(data)

    for key, value in stats.items():
        print(f'{key}: {value:.3}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


As we can see, the difference of the average order price between two restaurants is \\$ 1.44 and the food price of restaurant 1 is 0.16 cents more than the second restaurant, but the quantity of the food per order is higher in the second restaurant, which increases the average order price. The total shows that the average price is \\$ 43.6 and 10 food per order with \\$ 4.15 per food the average.

In [ ]:
print('\n1 - restaurant\n')
print_summary(orders[orders['restaurant']=='1 - restaurant'])
print('\n2 - restaurant\n')
print_summary(orders[orders['restaurant']=='2 - restaurant'])

print('\nTotal\n')
print_summary(orders)


1 - restaurant

avg_price_order: 29.5
avg_price_food: 4.25
count_food_orders: 6.94

2 - restaurant

avg_price_order: 31.0
avg_price_food: 4.09
count_food_orders: 7.57

Total

avg_price_order: 43.6
avg_price_food: 4.15
count_food_orders: 10.5


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


As we can see, the average frequency item is 509, that is, an item appears on average 509 times. When, we look at the median, we see that it is the outliers that pull the average up. The maximum value is 13093, which represents 56.8% of the orders with this item which is Pilau Rice. The frequency percentage of the item is 2.22% (mean / total unit orders) and the median percentage is 0.49%.

In [ ]:
items_frequence = orders.groupby('item_name').size().reset_index(name='quantity')
print(items_frequence.describe())
print(f'\ntotal unique orders: {len(orders["order_number"].unique())}')

           quantity
count    381.000000
mean     509.188976
std     1261.237290
min        1.000000
25%       26.000000
50%      113.000000
75%      408.000000
max    13093.000000

total unique orders: 23041


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We need to remove the extra spaces, symbols or accents if any, because maybe there are duplicate items but the same text with another way of writing.

In [ ]:
def text_normalization(text):
    new_text = ' '.join(text.split())
    return unidecode.unidecode(new_text.lower())

orders['item_name'] = orders['item_name'].apply(text_normalization)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<a id='market_basket_analysis'></a>
### Market Basket Analysis (Association Rules)

Through the frequency of the items in the orders, we try to understand if there is an association between one product and another in the order with the objective of recommending this product in an extra purchase.

### Example
We have a Market Basket transactions, where diapers and beer are a set of frequent items and with that, we have those who buy diapers have a probability of buying beer.

![example_mba_kaggle.png](attachment:7f09ccef-4191-4585-bc60-c1db7a9d03d5.png)

We need to transform the dataset into a list of items in order and convert it into a matrix (data set) where the columns are items and the rows are the order. If that item belongs to that order the value 1 is assigned, otherwise 0.

In [ ]:
#convert the dataframe to list items in order
item_list = orders.groupby('order_number')['item_name'].unique()

# transform the values of the data set to 1 if that item belongs to that order, otherwise 0
te = TransactionEncoder()
oht_orders = te.fit(item_list).transform(item_list, sparse=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



To save memory, we represent the transaction data in sparse format.  Because, we have 316 items and 23041 orders.

In [ ]:
sparse_df_items = pd.DataFrame.sparse.from_spmatrix(oht_orders, columns=te.columns_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



As a threshold for the minimum frequency of a set of items(the support metric), we used the percentage of the average/unique order frequency, which is 2.22% and max len of set of items equals 10.

In [ ]:
frequent_itemsets = apriori(sparse_df_items, min_support=0.02209,max_len=11, use_colnames=True, verbose=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Processing 600 combinations | Sampling itemset size 5


We create a copy of frequent item sets to create a custom output and analyze.

In [ ]:
frequent_itemsets_plot = frequent_itemsets.copy()
frequent_itemsets_plot['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets_plot['support'] = (frequent_itemsets_plot['support'] * 100).round(2)
frequent_itemsets_plot["itemsets"] = frequent_itemsets_plot["itemsets"].apply(lambda x: ', '.join(list(x))).astype("str")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



We get 556 frequency item sets with the filter we apply. We can see, that most of the item set sizes are 2 and the maximum value is 5. But it is not necessary that this itemset has a high frequency in orders. We have a high std for length equal to 1 indicating item variability and a low variability for length equal to 2 or 3 representing 4.64% and 3.48% respectively on average in the orders.

In [ ]:
frequent_itemsets_plot.groupby('length')['support'].describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,count,mean,std,min,25%,50%,75%,max
length,,,,,,,,
1,73.0,8.864384,8.976832,2.22,3.2800,5.06,10.580,50.17
2,252.0,4.637302,3.121391,2.21,2.7375,3.51,5.215,24.15
3,197.0,3.484112,1.451654,2.21,2.5400,2.99,3.920,12.07
4,43.0,2.908140,0.703607,2.27,2.4300,2.68,3.100,5.01
5,1.0,2.280000,NaN,2.28,2.2800,2.28,2.280,2.28


As we can see, 50% of orders have rice pilau and 39.78% have simple papadum and other information is 24.15% of orders have simple papadum and rice pilau in the same order and 19.2% have simple papadum, mango chutney. In addition, 2.28% of orders have simple papadum, onion chutney, mango chutney, mint sauce, rice pilau as items.

In [ ]:
top_20_frequence = frequent_itemsets_plot.sort_values('support',ascending=False).head(20).sort_values('support')
fig = px.bar(top_20_frequence, x="support", y="itemsets", orientation='h', text='support')
fig.update_traces(textposition="outside")
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We filter an lift equal to 1 to get only rules that have a probability of buying the antecedents and consequents in the same order. As we can see, we have a high probability of recomedations if the customer buys an item. For example, we have 174 possibilities of recommendations if the customer buys a plain papadum.

In [ ]:
market_basket_rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
market_basket_rules.groupby('antecedents').size().sort_values(ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



antecedents
(plain papadum)                                              174
(pilau rice)                                                 166
(mango chutney)                                              105
(naan)                                                        71
(mint sauce)                                                  70
                                                            ... 
(mint sauce, pilau rice, garlic naan)                          1
(garlic naan, mango chutney, pilau rice)                       1
(mint sauce, mango chutney, garlic naan)                       1
(garlic naan, naan, plain papadum)                             1
(mango chutney, onion chutney, pilau rice, plain papadum)      1
Length: 285, dtype: int64

<a id='item_recommendation'></a>
### Item Recommendation

To filter the best recommendations we will use the highest confidence value for each antecedent. We ran with the top 20 most frequent items and got some recommendations:

* 4.44% (confidence) of those who buys pilau rice, buys paratha as well;
* If one buys garlic naan, it's likely that one has also bought plain papadum with saag aloo. This way, it is possible to create a bundle with these items and apply a discount;
* If one buys pilau rice with chicken tikka masala it's likely that one has also bought madras. This way, it is possible apply discount in the madras.

In [ ]:
best_item_recommendations = market_basket_rules.sort_values(['confidence','lift'],ascending=False).drop_duplicates(subset=['antecedents'])
top_20_frequence_items = frequent_itemsets.sort_values('support',ascending=False).head(20)['itemsets']
best_item_recommendations[best_item_recommendations['antecedents'].isin(top_20_frequence_items)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
302,(mango chutney),(plain papadum),0.227811,0.397813,0.192006,0.842827,2.118654,0.101379,3.831372,0.683774
340,(mint sauce),(plain papadum),0.166225,0.397813,0.133414,0.802611,2.017560,0.067288,3.050764,0.604902
134,(chicken tikka masala),(pilau rice),0.231197,0.501714,0.158413,0.685189,1.365695,0.042419,1.582807,0.348297
418,(plain naan),(pilau rice),0.162840,0.501714,0.103728,0.636994,1.269634,0.022029,1.372664,0.253681
1283,"(mango chutney, plain papadum)",(pilau rice),0.192006,0.501714,0.120698,0.628617,1.252937,0.024366,1.341702,0.249848
39,(bombay aloo),(pilau rice),0.243522,0.501714,0.151165,0.620745,1.237248,0.028987,1.313854,0.253483
409,(peshwari naan),(pilau rice),0.155766,0.501714,0.096480,0.619393,1.234552,0.018330,1.309185,0.225044
368,(naan),(pilau rice),0.289875,0.501714,0.177640,0.612816,1.221445,0.032206,1.286949,0.255303
214,(keema naan),(pilau rice),0.160410,0.501714,0.097956,0.610660,1.217147,0.017476,1.279822,0.212493
421,(plain papadum),(pilau rice),0.397813,0.501714,0.241526,0.607135,1.210121,0.041938,1.268338,0.288343


<a id='conclusions'></a>
### Conclusions:

* The quantity of the food per order is higher in the second restaurant, which increases the average order price with difference \\$ 1.44;
* The average price is \\$ 43.6 and 10 food per order with \\$ 4.15 per food the average;
* The average frequency item is 509, that is, an item appears on average 509 times which represets 2.22%;
* 50,17% of orders has pilau rice;
* 24.15% of orders have simple papadum and rice pilau in the same order 19.2% have simple papadum, mango chutney;
* 2.28% of orders have simple papadum, onion chutney, mango chutney, mint sauce, rice pilau as items;
* 4.44% (confidence) of those who buys pilau rice, buys paratha as well;
* If one buys plain papadum with pilau rice it's likely that one has also bought garlic naan with naan;
* If one buys garlic naan, it's likely that one has also bought plain papadum with saag aloo. This way, it is possible to create a bundle with these items and apply a discount;
* If one buys pilau rice with chicken tikka masala it's likely that one has also bought madras. This way, it is possible apply discount in the madras.


And that’s it! It has been a pleasure to make this kernel, I have learned a lot! Thank you for reading and if you like it, please upvote it!

<a id='references'></a>
### References:

Annalyn Ng(2016) Association rules apriori algorithm tutorial. Retrieved from: [https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html](https://www.kdnuggets.com/2016/04/association-rules-apriori-algorithm-tutorial.html)

Hafsa Jabeen(2018) Market Basket Analysis using R. Retrieved from: [https://www.datacamp.com/community/tutorials/market-basket-analysis-r](https://www.datacamp.com/community/tutorials/market-basket-analysis-r)

Association Rules. Retrived from: [http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)

Apriori. Retrived from: [http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)
